In [12]:
import torch
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data



In [13]:
transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((28, 28)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [14]:
trainset = dset.ImageFolder(root="/Users/ilayda/Music/data/train_set/",transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,shuffle=True,  num_workers=1)

testset = dset.ImageFolder(root='/Users/ilayda/Music/data/test_set/',transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,shuffle=True,     num_workers=1)

In [15]:
classes=('low','medium','high')


In [16]:
# Hyper Parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 3
batch_size=100
num_epochs = 100
learning_rate = 0.01



In [17]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size,num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

rnn = RNN(input_size, hidden_size, num_layers, num_classes)




In [18]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)



In [19]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        labels -= 1
        images = Variable(images.view(-1, sequence_length, input_size))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if epoch % 1 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(trainset)//batch_size, loss.data[0]))



Epoch [1/100], Step [1/1], Loss: 1.0967
Epoch [1/100], Step [2/1], Loss: 1.3521
Epoch [2/100], Step [1/1], Loss: 1.1002
Epoch [2/100], Step [2/1], Loss: 1.0894
Epoch [3/100], Step [1/1], Loss: 1.1057
Epoch [3/100], Step [2/1], Loss: 1.1004
Epoch [4/100], Step [1/1], Loss: 1.0931
Epoch [4/100], Step [2/1], Loss: 1.1154
Epoch [5/100], Step [1/1], Loss: 1.1142
Epoch [5/100], Step [2/1], Loss: 1.0876
Epoch [6/100], Step [1/1], Loss: 1.0964
Epoch [6/100], Step [2/1], Loss: 1.1009
Epoch [7/100], Step [1/1], Loss: 1.1005
Epoch [7/100], Step [2/1], Loss: 1.0959
Epoch [8/100], Step [1/1], Loss: 1.1001
Epoch [8/100], Step [2/1], Loss: 1.0875
Epoch [9/100], Step [1/1], Loss: 1.0948
Epoch [9/100], Step [2/1], Loss: 1.1005
Epoch [10/100], Step [1/1], Loss: 1.1004
Epoch [10/100], Step [2/1], Loss: 1.0850
Epoch [11/100], Step [1/1], Loss: 1.0915
Epoch [11/100], Step [2/1], Loss: 1.0956
Epoch [12/100], Step [1/1], Loss: 1.0758
Epoch [12/100], Step [2/1], Loss: 1.0958
Epoch [13/100], Step [1/1], Loss: 

In [20]:
# Test the Model
correct = 0
total = 0
for images, labels in testloader:
    labels -= 1
    images = Variable(images.view(-1, sequence_length, input_size))
    
    
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 15 test images: %d %%' % (100 * correct / total)) 

# Save the Model
torch.save(rnn.state_dict(), 'rnn.pkl')

Test Accuracy of the model on the 15 test images: 26 %


In [10]:
outputs.shape

torch.Size([15, 3])

In [11]:
labels.shape

torch.Size([15])

In [12]:
labels.min()


0

In [13]:
labels.max()

2

In [14]:
classes[0]

'low'

In [15]:
classes[2]

'high'